## General principle of computing one point of the global MSL ##

The global Mean Sea Level is computed from the altimetric "reference mission", namely today (2023/04/19), Sentinel-6 Michael Freilich.
We won't compute the whole time series here, but show the basis of computing one point in this along-time data series.

<br>made for Eumetsat Copernicus Marine Data Service by V. Rosmorduc, CLS

** Import libraries **

In [1]:
# to interpret paths and folder for any OS
import os
import glob
import json
import shutil                                  # a library that allows us access to basic operating system commands
# math library
import numpy as np
# downloading / reading / using NetCDF data
import eumdac
import xarray as xr
# plotting library
import matplotlib
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
#statistical library
from scipy import stats
#remove warnings for ease of reading
import warnings
warnings.filterwarnings('ignore')

#we will look at the plot within the notebooks
%matplotlib inline

In [2]:
# Create a download directory for our products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

download_data = False

cycle = 53

In [3]:
if download_data:
# load credentials
    with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
        credentials = json.load(json_file)
        token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
        print(f"This token '{token}' expires {token.expiration}")

# create data store object
    datastore = eumdac.DataStore(token)

In [4]:
if download_data:
    #list Poseidon-4 collection IDs
    datastore = eumdac.DataStore(token)
    for collection_id in datastore.collections:
        if ("Poseidon-4" in collection_id.title):
            print(f"Collection ID({collection_id}): {collection_id.title}")

From the lists above, we will pick the "Poseidon-4 Altimetry Level 2 Low Resolution (baseline version FXX) - Sentinel-6 - Reprocessed" collection ID. (the baseline number is evolving with new reprocessing; if you have older files from another baseline (see at the end of the folder/file name, just before the .SEN6 or .nc), replace them with the newest, *_do not use inhomogeneous baselines_* in computations such as mean sea level)

To download only the files we'll be using, select only the "reduced" (RED) files only from the NTC delay ('NT') and download it. (NB using the Reprocessed product, they will always be "NTC", so there is no necessity for this condition in fact - just to show how it is done)

In [5]:
if download_data:
    collectionID = 'EO:EUM:DAT:0239'
    # space/time filter the collection for products
    selected_collection = datastore.get_collection(collectionID)

    products = selected_collection.search(
        cycle=cycle,
        timeliness="NT")

    print(f"Found {len(products)} products")

In [6]:
if download_data:
    for product in products:
        for entry in product.entries:
            if 'S6A_P4_2__LR_RED_' in entry:
                output_file = os.path.join(os.getcwd(), 'products',os.path.basename(entry))
                if not os.path.exists(output_file):
                    with product.open(entry=entry) as fsrc, open(output_file, mode='wb') as fdst:
                        print(f'Downloading {fsrc.name}.')
                        shutil.copyfileobj(fsrc, fdst)
                        print(f'Download of file {fsrc.name} finished.')

We read the data from a full cycle (track 1 to 254) from Sentinel-6 Reduced Non Time Critical level 2 Low resolution data.   
LR are more homogeneous with previous mission (and you don't need high resolution to compute a global mean sea level). 
NTC are more accurate (in particular relative to the orbit), and more homogeneous along the mission; when a reprocessing is done, it is done on the whole mission from its beginning, while NRT or STC processing or corrections can be changed without this applied to the oldest data. 
Reduced is because we don't need the extra information you have in the standard version.

reading all the files together + defining needed variables

In [7]:
s6_files = glob.glob(os.path.join(download_dir,'S6A_P4_2__LR_RED_*.nc'))
#we sort the files - by default it is "alphabetical" order, but pass numbers are also correctly ordered, which is our goal
# this sorting will provide with a well-orderer dataset/nc file (see next cell). 
# It is not absolutely vital to the present computation, but could be necessary in other cases
s6_files.sort()

if s6_files != []:
    s6cycle01 = xr.open_mfdataset(s6_files, group='data_01', combine='nested', concat_dim='time', compat='no_conflicts')
    s6cycle01ku = xr.open_mfdataset(s6_files, group='data_01/ku', combine='nested', concat_dim='time', compat='no_conflicts')
else:
    print('no files found')

#definition of the variables we will need
lat = s6cycle01['latitude']
lon = s6cycle01['longitude']
time = s6cycle01['time']
ssha = s6cycle01ku['ssha']

# use of the different flags ; 0 is for ocean (surface flag), no rain nor sea ice or for "good" (quality flags)
# alternate values are > 0, so if all the conditions are met, the sum will be zero.
flags_ssha = np.array(s6cycle01.surface_classification_flag) \
           + np.array(s6cycle01ku.range_ocean_qual) \
           + np.array(s6cycle01ku.off_nadir_angle_wf_ocean_qual) \
           + np.array(s6cycle01ku.sig0_ocean_qual) \
           + np.array(s6cycle01ku.swh_ocean_qual) \
           + np.array(s6cycle01.rain_flag) \
           + np.array(s6cycle01.meteo_map_availability_flag) \
           + np.array(s6cycle01.mean_sea_surface_sol1_qual) \
           + np.array(s6cycle01.rad_sea_ice_flag) \
           + np.array(s6cycle01.rad_wet_tropo_cor_qual)
ssha[flags_ssha != 0.0] = np.nan

# remove a few extreme outliers left
ssha = ssha.where((ssha > -2) & (ssha < 2))


OSError: [Errno -101] NetCDF: HDF error: b'/Users/benloveday/Code/Git_Reps/CMTS/internal/applications/ocean-case-studies/Case_studies/UN_Ocean_Decade/Challenge07_quantifying_sealevel_rise_with_S6/products/S6A_P4_2__LR_RED__NT_053_245_20220426T193100_20220426T202713_F06.nc'

In [ ]:
s6cycle01

Plotting the full cycle ; at this stage, we have along-track data with time as dimension. 
Note that this plot is taking time with 254 times about 3350 points to plot (**_this cell can be skipped with no impact on the following computations_**)

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.PlateCarree(), aspect=1.25)
ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
#zoom on the Mediterranean Western basin
#ax.set_extent([0, 25, 30, 50], crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k', zorder=0)
# actually plot the data
tracks = ax.scatter(lon, lat, c=ssha, cmap='RdYlBu_r', marker='o',  vmin=-0.25, vmax=0.25, edgecolors='black', linewidth=0.01)
cbar = plt.colorbar(tracks)
plt.title('Sea surface height with respect to a mean sea surface (SSHA) [m]', size=18)
plt.show()

We combine the datasets from the two groups in one, extracting only ssha (which is in the Ku group), and the coordinates/dimension (which are in the group above). We define the attributes which can be needed for an output NetCDF, so as to be able to use it as such afterwards (not necessary if you only want to compute the mean of the cycle).

In [ ]:
#creating a proper dataset with useful attributes for a nc files 
ds = xr.Dataset(
    {'ssha01': (['time'],  \
                ssha.data, \
                {'standard_name' : 'sea_surface_height_above_sea_level', \
                 'units' : "m", \
                 'coordinates' : "longitude latitude"})}, 
    coords={
        'time': (['time'], \
                 time.data, \
                 {'standard_name' : 'time', \
                  #'units' : "seconds since 2000-01-01 00:00:00.0", \ # a unit is kept in the concatenation by this dimension
                  'coordinates' : "longitude latitude"}),
        'lon01': (['time'], lon.data, \
                {'standard_name' : 'longitude', \
                 'units' : "degrees_east", \
                 'coordinates' : "longitude latitude"}), 
        'lat01': (['time'], lat.data, \
                  {'standard_name' : 'latitude', \
                   'units' : "degrees_north", \
                   'coordinates' : "longitude latitude"})})

# a few time steps are duplicated at the end of some passes and at the beggining of the following one. We are removing them.
## solution with an xarray version > 2022, to be tested 
#ds = ds.drop_duplicates(dim="time",  keep='first')
# solution with xarray versions before 2022: selection only non (~) duplicated times
ds = ds.sel(time=~ds.indexes['time'].duplicated())

ds.to_netcdf('sshafullcycle_'+str(cycle)+'.nc')
s6cycle01.close()
s6cycle01ku.close()
ds

Now we are "binning" the data in the two spatial dimensions, i.e. averaging them over boxes in longitude/latitude ; we do not take into account values at NaN (any arithmetic operation with a NaN in will result in NaN: we want to ignore those values when they are only part of the pixel - not when a pixel is always at NaN). However, note that due to the size of the pixels we are defining, a lot of pixels will be in part over lands. 

In [ ]:
# steps in degrees for the pixels we will create. 
# do not go much under those values, due to the gap between tracks in longitude (315 km at the Equator)
steplon = 3
steplat = 1
lat_min = -90. 
lon_min = 0.
lat_max = 90; lon_max = 360 

#creating the arrays of longitudes and latitudes
# NB. last point is not within the array in Python.
binnedlon = np.arange(lon_min,lon_max+steplon,steplon)
binnedlat = np.arange(lat_min,lat_max+steplat,steplat)
test = stats.binned_statistic_2d(ds.lon01, ds.lat01, ds.ssha01, statistic=np.nanmean,  bins=[binnedlon,binnedlat])


The binned_statistic-2d is outputing a (x,y) ndarray. However, in particular to plot the data on a map, you'll need a (lat,lon) matrix, so you have to "transpose" your ndarray (i.e. to switch dimensions).

Moreover, not to miss the 0 - 360 transition, we added one point to the statistics. 
We are choosing to reference the pixels ("registration") at their center ("pixel registration"), so we shift by half a pixel the lon/lat defining each pixel. The other usual solution is top left ("gridline registration").

In [ ]:
#defining an output xarray dataset
binnedgrid = xr.Dataset({
    'binnedssha': xr.DataArray(
        data = test.statistic.T,
        dims = ['latitude','longitude'],
        coords =  {'latitude': test.y_edge[:-1]+ steplat/2, 'longitude': test.x_edge[:-1] + steplon/2},
        attrs = {'long_name': 'sea surface heights anomalies by lat/lon bins', 'units': 'm', 'standard_name': 'sea_surface_height_above_sea_level'}
        )
    },
    #global attributes can be added
    attrs = { }
    )
binnedgrid

In [ ]:
# saving the output binned grid in a netCDF file
binnedgrid.to_netcdf('binnedgrid.nc')

In [ ]:
# plotting the resulting grid. It is a very pixellized map - which was the objective.

plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.))
# overlaying the continents on top makes a nicer plot (zorder continent > zorder SSHA).
# ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k', zorder=1)
ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k')
ax.set_extent([-180, 180, lat_min, lat_max])
mymap = ax.pcolor(binnedgrid.longitude, binnedgrid.latitude, binnedgrid.binnedssha, cmap='RdYlBu_r', vmin=-0.15, vmax=0.15, shading=None, zorder=0)
# Add a colorbar to the map object to give scale
plt.colorbar(mymap, ax=ax)
# Add a title to the map object
plt.title('binned Sea surface height with respect to a mean sea surface (SSHA) [m]', size=18)
plt.show()                  

Computing the global mean for the cycle. The area of each pixel is different depending on the latitude, 
so we weight our pixels with cos(latitude) so as not to overestimate the high latitude contributions.

One more refinement would be to weight each binned pixel with the amount of ocean within (i.e. not count land in the whole pixel "weight"; here we overweight coastal areas in the computation).


In [ ]:
weights = np.cos(np.deg2rad(binnedgrid['latitude']))
weights.name = "weights"
ssha_weighted = binnedgrid['binnedssha'].weighted(weights)
ssha_weighted
weighted_mean = ssha_weighted.mean(("longitude", "latitude"))
print("the Global mean sea level value (with respect to a 20-year mean) for this cycle is: ", np.around(weighted_mean.values,4), " meters")

This was an exercice to show how it is done. 

It is not, however, the whole processing to do to output a complete 30-year long time series showing sea level long-term changes and trend: once done on every single cycle (since Jan. 1993! - use harmonized data with homogeneous corrections and references). To have an interannual trend you need to remove seasonal cycles, filter for 60-day and six-month periodic variations. You also have intermission biases to apply when switching from one satellite to its follow-on. You can also (or not) remove a bias to take the Glacial Isostatic Adjustement into account (-0.3 mm/yr). See https://www.aviso.altimetry.fr/en/data/products/ocean-indicators-products/mean-sea-level/processing-and-corrections.html for the details of the complete computation of a mean sea level time series from the reference mission.

Beware that the zero level of this computation is *_not_* the one you can usually see on altimetry-derived mean sea level curves : here zero is set by construction at the 20-year mean, while most usually the zero is set at the level of the beginning of the series (in 1993).